In [ ]:
%matplotlib inline
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import cv2
import numpy as np
from glob import glob
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import pandas as pd
import json

In [ ]:
BASE_DATA_FOLDER = "../input/plant-seedlings-classification/"
TRAin_DATA_FOLDER = os.path.join(BASE_DATA_FOLDER, "train")

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
image_path = '../input/cassava-leaf-disease-classification/train_images'

In [ ]:
train.head()
label_to_disease = json.load(open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json'))
train['disease'] = train.label.map(label_to_disease)
label_to_disease

In [ ]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

In [ ]:
def visualize_scatter_with_images(X_2d_data, images, figsize=(45,45), image_zoom=1):
    fig, ax = plt.subplots(figsize=figsize)
    artists = []
    for xy, i in zip(X_2d_data, images):
        x0, y0 = xy
        img = OffsetImage(i, zoom=image_zoom)
        ab = AnnotationBbox(img, (x0, y0), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))
    ax.update_datalim(X_2d_data)
    ax.autoscale()
    plt.show()

In [ ]:
def visualize_scatter(data_2d, label_ids, figsize=(20,20)):
    plt.figure(figsize=figsize)
    plt.grid()
    
    nb_classes = len(np.unique(label_ids))
    
    for label_id in np.unique(label_ids):
        plt.scatter(data_2d[np.where(label_ids == label_id), 0],
                    data_2d[np.where(label_ids == label_id), 1],
                    marker='o',
                    color= plt.cm.Set1(label_id / float(nb_classes)),
                    linewidth='1',
                    alpha=0.8,
                    label=id_to_label_dict[label_id])
    plt.legend(loc='best')

In [ ]:
images = []
for image in os.listdir("../input/cassava-leaf-disease-classification/train_images/"):
    print(image)
#     for image_path in ):
    image = cv2.imread(os.path.join(image_path, image), cv2.IMREAD_COLOR)
    print(image)
    image = cv2.resize(image, (150, 150))
    image = segment_plant(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (45,45))

    image = image.flatten()

    images.append(image)
images = np.array(images)

In [ ]:
images_scaled = StandardScaler().fit_transform(images)

In [ ]:
images_scaled.shape

In [ ]:
plt.imshow(np.reshape(images[734], (45,45)), cmap="gray")

In [ ]:
pca = PCA(n_components=180)
pca_result = pca.fit_transform(images_scaled)

In [ ]:
pca_result.shape

In [ ]:
tsne = TSNE(n_components=2, perplexity=40.0)

In [ ]:
tsne_result = tsne.fit_transform(pca_result)

In [ ]:
tsne_result_scaled = StandardScaler().fit_transform(tsne_result)

In [ ]:
lab = list(label_to_disease.values())
lab

In [ ]:
labels = np.array(lab)
labels

In [ ]:
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [ ]:
label_ids = np.array(list([label_to_id_dict[x] for x in labels]))

In [ ]:
visualize_scatter(tsne_result_scaled, label_ids)

In [ ]:
visualize_scatter_with_images(tsne_result_scaled, images = [np.reshape(i, (45,45)) for i in images], image_zoom=0.7)

## 3D t-SNE with animation

In [ ]:
tsne = TSNE(n_components=3)

In [ ]:
# tsne_result = tsne.fit_transform(pca_result)

In [ ]:
tsne_result_scaled = StandardScaler().fit_transform(tsne_result)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

In [ ]:
fig = plt.figure(figsize=(25,25))
ax = fig.add_subplot(111,projection='3d')

plt.grid()
    
nb_classes = len(np.unique(label_ids))
    
for label_id in np.unique(label_ids):
    ax.scatter(tsne_result_scaled[np.where(label_ids == label_id), 0],
                tsne_result_scaled[np.where(label_ids == label_id), 1],
#                 tsne_result_scaled[np.where(label_ids == label_id), 2],
                alpha=0.8,
                color= plt.cm.Set1(label_id / float(nb_classes)),
                marker='o',
                label=id_to_label_dict[label_id])
ax.legend(loc='best')
ax.view_init(25, 45)
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)
ax.set_zlim(-2.5, 2.5)

In [ ]:
anim = animation.FuncAnimation(fig, lambda frame_number: ax.view_init(30, 4 * frame_number), interval=75, frames=90)

In [ ]:
plot_3d_animation_filename = 'animation.gif'
anim.save(plot_3d_animation_filename, writer='imagemagick')

In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open(plot_3d_animation_filename, 'r+b').read()
# HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(base64.b64encode(video).decode('ascii')))

With this last step we would get a neat little 3D scatter plot animation. I commented it, because the size of the gif can take up to 30MB and it slows down the whole notebook unfortunately. If you would like to try it out, just uncomment it and run the last cell. Or of course you can view it with any program because it is saved as `animation.gif` 